In [107]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_hub as  hub
import tensorflow_datasets as tfds
import  numpy as np
import mtcnn
from mtcnn import MTCNN
import PIL
from PIL import Image
import pandas as pd
from IPython.display import display
import os
import matplotlib.pyplot as pyplot
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import time
import joblib

In [165]:
model = tf.keras.models.load_model('../Transfer learning/face-recogition-colab.h5',
                                   custom_objects={'KerasLayer':hub.KerasLayer})
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1001)              55875273  
_________________________________________________________________
dense (Dense)                (None, 7)                 7014      
_________________________________________________________________
activation (Activation)      (None, 7)                 0         
Total params: 55,882,287
Trainable params: 7,014
Non-trainable params: 55,875,273
_________________________________________________________________


In [166]:
detector = MTCNN()

In [167]:
names = joblib.load('../Training Data Creation/label.pkl')
names

['Aditya Solanki',
 'Ben Afflek',
 'Elton John',
 'Jerry Seinfeld',
 'Madonna',
 'Mindy Kaling',
 'Unknown']

In [168]:
def predict_face(model, filename, detector):
    image = Image.open(filename)
    #Convert the file to RGB
    image = image.convert('RGB')
    #Convert the File to Numpy array to be machine readable
    pixels = np.asarray(image)
    
    #Extracting Face embeddings from the Photo
    result = detector.detect_faces(pixels)

    x1, y1, width, height = result[0]['box']
    x1, y1 = abs(x1) , abs(y1)
    x2, y2 = abs(x1) + width , abs(y1) + height 
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize((299,299))
    face_array = np.asarray(image)
    #face_pixels = face_array.astype('float32')
    #mean, std = face_pixels.mean(), face_pixels.std()
    #face_pixels  = (face_pixels - mean)/std
    face_pixels  = face_array / 255
    samples = np.expand_dims(face_pixels, axis = 0)
    prediction = model.predict(samples)
    return prediction[0]

In [210]:
filename = ('../data/test/Elton.jpg')
prediction = predict_face(model, filename,detector)

In [211]:
list(prediction)

[0.98615277,
 2.2895051e-07,
 9.68683e-05,
 1.7410703e-06,
 1.1138621e-06,
 9.202803e-07,
 0.013746338]

In [212]:
predict = dict(zip(names, prediction))
predict_reverse = dict(zip(prediction, names))
predict_reverse[max(predict.values())]

'Aditya Solanki'